In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
import time

from copy import deepcopy
from numpy import diag, exp, log, sqrt, trace
from numpy.linalg import eig, inv, norm, solve, svd
from scipy.linalg import expm, solve_lyapunov, solve_sylvester, sqrtm
from scipy.sparse import diags

$$\varphi_j'(t) = \alpha \varphi_j(t) + \beta \varphi_{j+1}(t-\tau_{j+1}), \qquad j=1,\dots,M$$

In [ ]:
def history_function(t):
    return 0.0
    #return np.sin(0.1 * t)

d = 50
alpha = -1.2  # -0.35
beta = 1
tau = 0.1 * np.ones(d) # np.arange(d) / 5
T_end = 2.0
delta_t = 0.01
N_end = int(np.ceil(T_end / delta_t))
X_0 = 0.5 * np.random.randn(d)

A = diag([alpha] * d) # + 0.01 * np.random.randn(d)
N = diags(beta * np.ones(d - 1), 1).toarray()
N[d - 1, 0] = beta
C = np.eye(d) # np.diag([1] * 5 + [0] * (d - 5))
B_in = X_0[:, np.newaxis].dot(X_0[:, np.newaxis].T)

if ~np.all(np.real(eig(A)[0]) < 0):
    print('not all eigenvalues are negative')

X = np.zeros([d, N_end + 1])
X[:, 0] = X_0

for n in range(N_end):
    # code for diagonal A
    #X[:, n + 1] = X[:, n] + alpha * X[:, n] * delta_t
    #n_minus_tau = np.maximum(n - np.ceil(tau[np.concatenate([np.arange(1, d), np.array([0])])] / delta_t).astype(int), 0)
    #X[n_minus_tau != 0, n + 1] += beta * X[np.concatenate([np.arange(1, d), np.array([0])]), n_minus_tau][n_minus_tau != 0] * delta_t
    #X[n_minus_tau == 0, n + 1] += history_function(n * delta_t) * np.ones(sum(n_minus_tau == 0)) * delta_t

    n_minus_tau = np.maximum(n - np.ceil(tau[np.concatenate([np.arange(1, d), np.array([0])])] / delta_t).astype(int), 0)
    X[n_minus_tau != 0, n + 1] = (X[n_minus_tau != 0, n]
                                        + A.dot(X[:, n])[n_minus_tau != 0] * delta_t 
                                        + N.dot(X[np.arange(d), n_minus_tau])[n_minus_tau != 0] * delta_t)
    X[n_minus_tau == 0, n + 1] = (X[n_minus_tau == 0, n]
                                        + A.dot(X[:, n])[n_minus_tau == 0] * delta_t 
                                        + history_function(n * delta_t) * np.ones(sum(n_minus_tau == 0)) * delta_t)
    
plt.plot(np.linspace(0, T_end, N_end + 1), X.T);

In [ ]:
# check condition < 1
np.sqrt(T_end) * max(np.real(eig(N)[0])) / np.sqrt(2 * np.abs(np.max(eig(A)[0])))

In [ ]:
I = 2 * d
Q_i = np.zeros([I + 1, d, d])
Q_i[0, :, :] = solve_lyapunov(A.T, -C.T.dot(C))
for i in range(I):
    Q_i[i + 1, :, :] = solve_lyapunov(A.T, -N.T.dot(Q_i[i, :, :]).dot(N))
Q = np.sum(Q_i, 0)

I = 2 * d
P_i = np.zeros([I + 1, d, d])
P_i[0, :, :] = solve_lyapunov(A, -B_in.dot(B_in.T))
for i in range(I):
    P_i[i + 1, :, :] = solve_lyapunov(A, -N.dot(P_i[i, :, :]).dot(N.T))
P = np.sum(P_i, 0)

if np.sum(P_i[-1, :, :]) > 1e-6 or np.sum(Q_i[-1, :, :]) > 1e-6:
    print('Lyapunov solver did not converge')

#K = sqrtm(P)
#L = sqrtm(Q)

V_P, sigmas_P, U_P_T = svd(P)
K = V_P.dot(sqrt(diag(sigmas_P))).dot(U_P_T)

V_Q, sigmas_Q, U_Q_T = svd(Q)
L = V_Q.dot(sqrt(diag(sigmas_Q))).dot(U_Q_T)

V, sigmas, U_T = svd(K.T.dot(L))
Sigma = np.diag(sigmas)
Sigma_inv_05 = np.diag(1 / sqrt(sigmas))

T = Sigma_inv_05.dot(U_T).dot(L.T)
T_inv = K.dot(V).dot(Sigma_inv_05)
A_ = T.dot(A).dot(T_inv)
N_ = T.dot(N).dot(T_inv)
C_ = C.dot(T_inv)

plt.scatter(range(1, d + 1), sigmas) # (Hankel singular values)
plt.yscale('log');
plt.ylim(0.8 * min(sigmas), 1.2 * max(sigmas));

In [ ]:
r = 2

A_11 = A_[:r, :r]
A_12 = A_[:r, r:]
A_21 = A_[r:, :r]
A_22 = A_[r:, r:]
N_11 = N_[:r, :r]
N_12 = N_[:r, r:]
N_21 = N_[r:, :r]
N_22 = N_[r:, r:]
C_1 = C_[:, :r]
C_2 = C_[:, r:]

# Balanced truncation
A_r = A_11
N_r = N_11
C_r = C_1

X = np.zeros([d, N_end + 1])
X_r = np.zeros([r, N_end + 1])
X[:, 0] = X_0
X_r[:, 0] = T.dot(X_0)[:r]

for n in range(N_end):
    n_minus_tau = np.maximum(n - np.ceil(tau[np.concatenate([np.arange(1, d), np.array([0])])] / delta_t).astype(int), 0)

    X[n_minus_tau != 0, n + 1] = (X[n_minus_tau != 0, n]
                                        + A.dot(X[:, n])[n_minus_tau != 0] * delta_t 
                                        + N.dot(X[np.arange(d), n_minus_tau])[n_minus_tau != 0] * delta_t)
    X[n_minus_tau == 0, n + 1] = (X[n_minus_tau == 0, n]
                                        + A.dot(X[:, n])[n_minus_tau == 0] * delta_t 
                                        + history_function(n * delta_t) * np.ones(sum(n_minus_tau == 0)) * delta_t)
   
    X_r[n_minus_tau[:r] != 0, n + 1] = (X_r[n_minus_tau[:r] != 0, n]
                                        + A_r.dot(X_r[:, n])[n_minus_tau[:r] != 0] * delta_t 
                                        + N_r.dot(X_r[np.arange(r), n_minus_tau[:r]])[n_minus_tau[:r] != 0] * delta_t)
    X_r[n_minus_tau[:r] == 0, n + 1] = (X_r[n_minus_tau[:r] == 0, n]
                                        + A_r.dot(X_r[:, n])[n_minus_tau[:r] == 0] * delta_t 
                                        + history_function(n * delta_t) * np.ones(sum(n_minus_tau[:r] == 0)) * delta_t)


Y = C.dot(X)
Y_r = C_r.dot(X_r)

In [ ]:
t_val = np.linspace(0, T_end, N_end + 1)

COLORS = ['tab:blue', 'tab:orange', 'tab:green', 'tab:red', 'tab:purple', 'tab:olive',
          'tab:pink', 'tab:gray', 'tab:brown', 'tab:cyan']

plt.title(r'$d = %d, r = %d, \Delta t = %.1e$' % (d, r, delta_t))
for i, j in enumerate([0, 1, 2, 3, 4, 5]):#enumerate(np.random.choice(range(d), 3, replace=False)): # add without replacement!
    plt.plot(t_val, Y[j, :], '--', color=COLORS[i])
    plt.plot(t_val, Y_r[j, :], color=COLORS[i]);

## vary r, compute bounds

In [ ]:
L2_errors = []
bounds_thm2 = []
    
for r in range(1, d):
    A_11 = A_[:r, :r]
    A_12 = A_[:r, r:]
    A_21 = A_[r:, :r]
    A_22 = A_[r:, r:]
    N_11 = N_[:r, :r]
    N_12 = N_[:r, r:]
    N_21 = N_[r:, :r]
    N_22 = N_[r:, r:]
    C_1 = C_[:, :r]
    C_2 = C_[:, r:]

    # Balanced truncation
    A_r = A_11 # np.real(A_11)
    N_r = N_11 # np.real(N_11)
    C_r = C_1 # np.real(C_1)
    
    X = np.zeros([d, N_end + 1])
    X_r = np.zeros([r, N_end + 1])
    X[:, 0] = X_0
    X_r[:, 0] = T.dot(X_0)[:r]

    for n in range(N_end):
        n_minus_tau = np.maximum(n - np.ceil(tau[np.concatenate([np.arange(1, d), np.array([0])])] / delta_t).astype(int), 0)
        #X[n_minus_tau != 0, n + 1] += X[np.concatenate([np.arange(1, d), np.array([0])]), n_minus_tau][n_minus_tau != 0] * delta_t
        #X[n_minus_tau == 0, n + 1] += history_function(n * delta_t) * np.ones(sum(n_minus_tau == 0)) * delta_t

        #X_r[n_minus_tau[:r] != 0, n + 1] = (X_r[n_minus_tau[:r] != 0, n]
        #                                    + A_r.dot(X_r[:, n])[n_minus_tau[:r] != 0] * delta_t 
        #                                    + N_r.dot(X_r[np.arange(0, r), n_minus_tau[:r]])[n_minus_tau[:r] != 0] * delta_t)
        #X_r[n_minus_tau[:r] == 0, n + 1] = (X_r[n_minus_tau[:r] == 0, n]
        #                                    + A_r.dot(X_r[:, n])[n_minus_tau[:r] == 0] * delta_t 
        #                                    + history_function(n * delta_t) * np.ones(sum(n_minus_tau[:r] == 0)) * delta_t)


        X[n_minus_tau != 0, n + 1] = (X[n_minus_tau != 0, n]
                                            + A.dot(X[:, n])[n_minus_tau != 0] * delta_t 
                                            + N.dot(X[np.arange(d), n_minus_tau])[n_minus_tau != 0] * delta_t)
        X[n_minus_tau == 0, n + 1] = (X[n_minus_tau == 0, n]
                                            + A.dot(X[:, n])[n_minus_tau == 0] * delta_t 
                                            + history_function(n * delta_t) * np.ones(sum(n_minus_tau == 0)) * delta_t)

        X_r[n_minus_tau[:r] != 0, n + 1] = (X_r[n_minus_tau[:r] != 0, n]
                                            + A_r.dot(X_r[:, n])[n_minus_tau[:r] != 0] * delta_t 
                                            + N_r.dot(X_r[np.arange(r), n_minus_tau[:r]])[n_minus_tau[:r] != 0] * delta_t)
        X_r[n_minus_tau[:r] == 0, n + 1] = (X_r[n_minus_tau[:r] == 0, n]
                                            + A_r.dot(X_r[:, n])[n_minus_tau[:r] == 0] * delta_t 
                                            + history_function(n * delta_t) * np.ones(sum(n_minus_tau[:r] == 0)) * delta_t)

    Y = C.dot(X)
    Y_r = C_r.dot(X_r)
    
    L2_error = np.sqrt(np.sum(np.sum((Y - np.real(Y_r))**2, 0)) * delta_t)
    L2_errors.append(L2_error)
    
    A_hat = np.zeros([d + r, d + r])
    A_hat[0:d, 0:d] = A
    A_hat[d:d + r, d:d + r] = A_r
    B_in_hat = np.concatenate([B_in, T.dot(B_in)[:r, :]])
    C_hat = np.concatenate([C, -C_r], axis=1)
    N_hat = np.zeros([d + r, d + r])
    N_hat[0:d, 0:d] = np.sqrt(T_end) * N
    N_hat[d:d + r, d:d + r] = np.sqrt(T_end) * N_r

    I = 1500
    Q_hat_i = np.zeros([I + 1, d + r, d + r])
    Q_hat_i[0, :, :] = solve_lyapunov(A_hat.T, -C_hat.T.dot(C_hat))
    for i in range(I):
        Q_hat_i[i + 1, :, :] = solve_lyapunov(A_hat.T, -N_hat.T.dot(Q_hat_i[i, :, :]).dot(N_hat))
        if np.sum(Q_hat_i[i + 1, :, :]) > 1e20:
            print('Q has too high numbers')
            break
        if np.sum(Q_hat_i[i + 1, :, :]) < 1e-6:
            break
    Q_hat = np.sum(Q_hat_i, 0)
    
    if np.sum(Q_hat_i[-1, :, :]) > 1e-6:
        print('Lyapunov solver did not converge')

    I = 1500
    P_hat_i = np.zeros([I + 1, d + r, d + r])
    P_hat_i[0, :, :] = solve_lyapunov(A_hat, -B_in_hat.dot(B_in_hat.T))
    for i in range(I):
        P_hat_i[i + 1, :, :] = solve_lyapunov(A_hat, -N_hat.dot(P_hat_i[i, :, :]).dot(N_hat.T))
        if np.sum(P_hat_i[i + 1, :, :]) > 1e20:
            print('P has too high numbers')
            break
        if np.sum(P_hat_i[i + 1, :, :]) < 1e-6:
            break
    P_hat = np.sum(P_hat_i, 0)
    
    if np.sum(P_hat_i[i + 1, :, :]) > 1e-6:
        print('Lyapunov solver did not converge')

    #K_hat = sqrtm(P_hat)
    #L_hat = sqrtm(Q_hat)

    V_P_hat, sigmas_P_hat, U_P_hat_T = svd(P_hat)
    K_hat = V_P_hat.dot(sqrt(diag(sigmas_P_hat))).dot(U_P_hat_T)

    V_Q_hat, sigmas_Q_hat, U_Q_hat_T = svd(Q_hat)
    L_hat = V_Q_hat.dot(sqrt(diag(sigmas_Q_hat))).dot(U_Q_hat_T)

    V_hat, sigmas_hat, U_T_hat = svd(K_hat.T.dot(L_hat))
    Sigma_hat = np.diag(sigmas_hat)
    Sigma_inv_05 = np.diag(1 / sqrt(sigmas_hat))
    
    bounds_thm2.append(4 * sum(sigmas_hat) * np.sqrt(np.sum(X_0**2)))


In [ ]:
# check condition < 1
np.sqrt(T_end) * max(np.real(eig(N_hat)[0])) / np.sqrt(2 * np.abs(np.max(eig(A)[0])))

In [ ]:
r = 6

A_11 = A_[:r, :r]
A_12 = A_[:r, r:]
A_21 = A_[r:, :r]
A_22 = A_[r:, r:]
N_11 = N_[:r, :r]
N_12 = N_[:r, r:]
N_21 = N_[r:, :r]
N_22 = N_[r:, r:]
C_1 = C_[:, :r]
C_2 = C_[:, r:]

# Balanced truncation
A_r = A_11
N_r = N_11
C_r = C_1

X = np.zeros([d, N_end + 1])
X_r = np.zeros([r, N_end + 1])
X[:, 0] = X_0
X_r[:, 0] = T.dot(X_0)[:r]

for n in range(N_end):
    n_minus_tau = np.maximum(n - np.ceil(tau[np.concatenate([np.arange(1, d), np.array([0])])] / delta_t).astype(int), 0)

    X[n_minus_tau != 0, n + 1] = (X[n_minus_tau != 0, n]
                                        + A.dot(X[:, n])[n_minus_tau != 0] * delta_t 
                                        + N.dot(X[np.arange(d), n_minus_tau])[n_minus_tau != 0] * delta_t)
    X[n_minus_tau == 0, n + 1] = (X[n_minus_tau == 0, n]
                                        + A.dot(X[:, n])[n_minus_tau == 0] * delta_t 
                                        + history_function(n * delta_t) * np.ones(sum(n_minus_tau == 0)) * delta_t)
   
    X_r[n_minus_tau[:r] != 0, n + 1] = (X_r[n_minus_tau[:r] != 0, n]
                                        + A_r.dot(X_r[:, n])[n_minus_tau[:r] != 0] * delta_t 
                                        + N_r.dot(X_r[np.arange(r), n_minus_tau[:r]])[n_minus_tau[:r] != 0] * delta_t)
    X_r[n_minus_tau[:r] == 0, n + 1] = (X_r[n_minus_tau[:r] == 0, n]
                                        + A_r.dot(X_r[:, n])[n_minus_tau[:r] == 0] * delta_t 
                                        + history_function(n * delta_t) * np.ones(sum(n_minus_tau[:r] == 0)) * delta_t)


Y_2 = deepcopy(C.dot(X))
Y_2_r = deepcopy(C_r.dot(X_r))

In [ ]:
COLORS = ['tab:blue', 'tab:orange', 'tab:green', 'tab:red', 'tab:purple', 'tab:olive',
          'tab:pink', 'tab:gray', 'tab:brown', 'tab:cyan']

fig, ax = plt.subplots(1, 3, figsize=(15, 3.5))
ax[0].set_title('Errors of reduced systems')
ax[0].scatter(np.arange(1, d), L2_errors, label=r'$L^2$ error')
ax[0].scatter(np.arange(1, d), bounds_thm2, label='bound')
ax[0].set_yscale('log')
ax[0].set_ylim(0.5 * min(L2_errors), 2 * max(bounds_thm2));
ax[0].set_xlabel(r'$r$')
ax[0].legend()

t_val = np.linspace(0, T_end, N_end + 1)

ax[1].set_title(r'Full and reduced trajectories, $d = %d, r = %d$' % (d, 2))
for i, j in enumerate([0, 1, 2, 3, 4, 5]):
    ax[1].plot(t_val, Y[j, :], color=COLORS[i])
    ax[1].plot(t_val, Y_r[j, :], '--', color=COLORS[i]);
ax[1].set_xlabel(r'$t$')

ax[2].set_title(r'Full and reduced trajectories, $d = %d, r = %d$' % (d, 6))
for i, j in enumerate([0, 1, 2, 3, 4, 5]):
    ax[2].plot(t_val, Y_2[j, :], color=COLORS[i])
    ax[2].plot(t_val, Y_2_r[j, :], '--', color=COLORS[i]);
ax[2].set_xlabel(r'$t$');
    
fig.savefig('img/stuart_landau_d50_T2.pdf')